# Description du Script

## Fonctionnalité
Ce script implémente une interface utilisateur interactive à l'aide de **ipywidgets** dans un Jupyter Notebook. Il est conçu pour prendre en entrée un texte utilisateur (tweet), puis une prédiction est retourné via l'API.

## Détails Techniques
1. **Bibliothèques Utilisées**:
   - `ipywidgets`: pour créer des widgets interactifs comme des champs de texte, des boutons, etc.
   - `requests`: pour effectuer des requêtes HTTP, probablement vers une API externe.

2. **Widgets Décrits**:
   - Un champ de texte interactif pour entrer un texte (un tweet).
   - Un bouton pour déclencher une action (un envoi).

3. **Action Prédite**:
   - Lors de l'appui sur le bouton, le script envoye le contenu du champ de texte à l'API via une requête HTTP pour obtenir une réponse (prédiction ).

4. **Interface Utilisateur**:
   - L'interface repose sur des composants visuels pour faciliter l'interaction avec l'utilisateur.
   - Les résultats de l'action (prédiction) sont affichés dans le Notebook.


In [1]:
# Importer les modules nécessaires
import ipywidgets as widgets
from IPython.display import display
import requests

In [ ]:
# Importer les modules nécessaires
import ipywidgets as widgets
from IPython.display import display
import requests

# Fonction pour envoyer une requête POST à l'API et obtenir la prédiction
def test_api(tweet):
    url = "http://myapiapp-projet7.francecentral.azurecontainer.io:8000/predict"

    try:
        # Envoi de la requête POST avec le texte du tweet
        response = requests.post(url, json={"text": tweet})
        
        # Vérification du statut de la réponse
        if response.status_code == 200:
            # Extraire la prédiction de la réponse JSON
            prediction = response.json().get("predictions", ["Erreur dans la prédiction"])[0]
            display(widgets.HTML(value=f"<b>Prédiction : {prediction}</b>"))
            return prediction
        else:
            display(widgets.HTML(value=f"<b>Erreur : {response.json().get('error', 'Erreur inconnue')}</b>"))
            return None
    except Exception as e:
        display(widgets.HTML(value=f"<b>Erreur de connexion : {e}</b>"))
        return None

# Fonction pour envoyer un feedback de prédiction incorrecte
def send_feedback(tweet, prediction, feedback_type="non_valide"):
    #url = "http://api-prediction:8123/feedback"  # URL de l'API pour envoyer un feedback
    url = "http://myapiapp-projet7.francecentral.azurecontainer.io:8000/feedback"
    
    try:
        # Envoi de la requête POST avec le texte du tweet, la prédiction et le type de feedback
        response = requests.post(url, json={"text": tweet, "prediction": prediction, "feedback": feedback_type})
        
        # Vérification du statut de la réponse
        if response.status_code == 200:
            print("Feedback envoyé avec succès.")
        else:
            print("Erreur lors de l'envoi du feedback :", response.json().get("error", "Erreur inconnue"))
    except Exception as e:
        print("Erreur de connexion pour l'envoi du feedback :", e)

# Interface pour la saisie du tweet
tweet_input = widgets.Text(
    placeholder='Entrez un tweet ici',
    description='Tweet:',
    disabled=False
)

# Bouton de validation pour la prédiction
validation_button = widgets.Button(description="Valider la Prédiction")
non_validation_button = widgets.Button(description="Non Valide")

# Conteneur pour afficher la prédiction
output = widgets.Output()

# Fonction de gestion du clic sur le bouton de validation
def on_validation_click(b):
    with output:
        output.clear_output()
        print("Prédiction validée par l'utilisateur.")
        
        # Envoyer un feedback positif à l'API
        send_feedback(tweet_input.value, last_prediction, feedback_type="valide")

# Fonction de gestion du clic en cas de non-validation
def on_non_validation_click(b):
    with output:
        output.clear_output()
        print("Prédiction non validée par l'utilisateur.")
        
        # Envoyer un feedback négatif à l'API
        send_feedback(tweet_input.value, last_prediction, feedback_type="non_valide")

# Liaison des boutons avec les fonctions de gestion
validation_button.on_click(on_validation_click)
non_validation_button.on_click(on_non_validation_click)

# Fonction d'affichage des widgets
def display_interface(_):
    global last_prediction
    with output:
        output.clear_output()
        last_prediction = test_api(tweet_input.value)  # Stocker la dernière prédiction
        if last_prediction is not None:
            display(validation_button, non_validation_button)

# Bouton de lancement de la prédiction
predict_button = widgets.Button(description="Prédire")
predict_button.on_click(display_interface)

# Affichage des widgets
display(tweet_input, predict_button, output)


Text(value='', description='Tweet:', placeholder='Entrez un tweet ici')

Button(description='Prédire', style=ButtonStyle())

Output()